<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/network_connect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cdlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 228 kB 4.9 MB/s 
     |████████████████████████████████| 14.3 MB 5.0 MB/s 
     |████████████████████████████████| 2.6 MB 33.3 MB/s 
     |████████████████████████████████| 3.2 MB 36.8 MB/s 
     |████████████████████████████████| 175 kB 19.4 MB/s 
     |████████████████████████████████| 2.2 MB 14.0 MB/s 
  Created wheel for pyclustering: filename=pyclustering-0.10.1.2-py3-none-any.whl size=2395122 sha256=137145755b6e702292ab47b4683b1a82c555aca6c646f2f99ea17534f28d02b0
  Stored in directory: /root/.cache/pip/wheels/ea/87/6b/1e0568b5ba9dc6518a25338bae90bd8392f35206bb90bb10f1
  Created wheel for python-igraph: filename=python_igraph-0.10.2-py3-none-any.whl size=9074 sha256=4af8b91a02a450b20239c0a5ca7000917da777e0313f0470d12562b39a8fcc7e
  Stored in directory: /root/.cache/pip/wheels/74/1d/33/83ec153aa37c92957b2edb888f4f42fdea9b7ac89f4b919391
Successfu

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [31]:
%reset -f

# 1.0 Call libraries
import pandas as pd
import numpy as np
import networkx as nx

# 1.01
import joblib
from collections import defaultdict

# 1.02
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# 1.08 Misc
import os,gc , time,  glob
from os.path import exists
import feather
import re
import ast


In [32]:
# Paths:
modulePath = "/gdrive/MyDrive/Colab_data_files/talkingData/03112022/"
dataPath = "/gdrive/MyDrive/Colab_data_files/skipgram/"
modelsPath = "/gdrive/MyDrive/Colab_data_files/skipgram/modelspath/"
pathToStoreProgress = "/gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/"
mapdictPath = "/gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/"

In [33]:
# 1.09 Home made modules
os.chdir(modulePath)
import network_features
from network_features import *
import utils
from utils import *

import importlib; importlib.reload(network_features)
import importlib; importlib.reload(utils)

<module 'utils' from '/gdrive/MyDrive/Colab_data_files/talkingData/03112022/utils.py'>

In [34]:
# Cleaning up
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/modelspath/
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/

In [7]:
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/vectors.tsv
!rm -r -f /gdrive/MyDrive/Colab_data_files/skipgram/metadata.tsv

In [35]:

!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/modelspath/
!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/pathToStoreProgress/
!mkdir /gdrive/MyDrive/Colab_data_files/skipgram/mapdictpath/

In [36]:
# 8.0 Read back pkl file and dictionary:
os.chdir(dataPath)
train = pd.read_pickle("seq.pkl")
train.head()

,a,b
0,174,2146
1,3743,3067
2,9,917
3,14,759
4,1283,1173


In [ ]:
# Read dictionary
os.chdir(dataPath)
filehandler = open("word_index.txt", 'r')
word2index = filehandler.read()
word2index

In [38]:
org_columns = train.columns
cat_cols = ['a', 'b']

In [39]:
# 6.1 Create new columns (combinations of columns) in train
train = utils.create_cat_pairwise_cols(train , cat_cols)

In [40]:
train.head()

,a,b,a_p_b
0,174,2146,174_2146
1,3743,3067,3743_3067
2,9,917,9_917
3,14,759,14_759
4,1283,1173,1283_1173


In [ ]:
# 6.11 The train columns are:
current_train_columns = train.columns
# 6.12 New columns created. These columns must be latter dropped
new_cols = list(set(current_train_columns).difference(set(org_columns)))
# 6.13 Revised cat cols (ie excluding hour etc)
cat_cols.extend(new_cols)
len(cat_cols)   # 15
cat_cols

In [42]:
# 8.1 Get all possibe permutation of our columns
#     Also decide which all centrality measures to cal
deg = 1    # Calculate degree centrality
eig = 1    # Calculate eigenvector centrality
pr =  1    # Calculate pagerank centrality
clu = 1    # Clustering coeff centrality
nf = nx.katz_centrality_numpy # nx.katz_centrality_numpy
cMeasures = [deg,eig,pr,clu, nf]

In [ ]:
single_permute_list = utils.permute_colList(cat_cols,cMeasures)
single_permute_list

In [44]:
len(single_permute_list)   # 6

6

In [ ]:
spl = pd.DataFrame(single_permute_list, columns = ["colToProject", "intermediaryCol", "cMeasures?"])
spl.shape   # (210, 4)
spl

In [46]:
X = train

In [ ]:
utils.ulevelsPerThreshold(X,'a_p_b', threshold_list = [0.00001, 0.0001, 0.0002, 0.000485,0.0005, 0.001, 0.01, 0.1, 0.15, 0.2])

In [ ]:
mergethreshold = 0.00050  # 0.0020
X =  utils.levelsTransformer(X, 'a_p_b',mergethreshold, mapdictPath)

No saved dictionary exists as yet.


In [48]:
record_sets = list(spl.to_records(index = False))
record_sets

[('a', 'b', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>])),
 ('a', 'a_p_b', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>])),
 ('b', 'a', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>])),
 ('b', 'a_p_b', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>])),
 ('a_p_b', 'a', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>])),
 ('a_p_b', 'b', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>]))]

In [49]:
record_sets = record_sets[:4]

In [50]:
fulllist = [record_sets]

In [51]:
m = 1
for k,j in enumerate(fulllist):
  print("Record set no is {} of {}th fulllist ".format(k+1,m+1))
  print("Set of records is: ", j)
  # Each outer loop creates a new sets of models
  fulllist_repeat_no = m + 1
  transYesNo = False  # True implies transformation
  X1_train, X1_test = network_features.get_store_features(X,j, transYesNo, None, fulllist_repeat_no , modelsPath, pathToStoreProgress)
  print("\nList done. X1_train shape ", X1_train.shape)
  del X1_train
  gc.collect()

Record set no is 1 of 2th fulllist 
Set of records is:  [('a', 'b', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>])), ('a', 'a_p_b', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>])), ('b', 'a', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>])), ('b', 'a_p_b', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>]))]
('a', 'b', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>]))

Next:  1 a b
No of unique values in  a  are:  4044
Bipartite Network created. Time taken:  0.023650991916656493 minutes
Bipartite Network projected. Time taken:  0.3831348816553752 minutes
Model extracted. Time taken:  0.642585571606954 minutes
Model stored to disk
train columns:  3
No test data
Done with a b
Time taken:  1.0690585494041442
Entry made in file progress.csv
('a', 'a_p_b', list([1, 1, 1, 1, <function katz_centrality_numpy at 0x7ff0bb6b5440>]))

Next:  2 a a_p_b
No of unique values in  a  are:  40

In [ ]:
# Update levels or merge rarely occurring levels
#  into one. Dictionaries for performing this mapping
#   are in folder: mapdictpath
X = utils.updateLevels(X,
                        mapdictPath,
                        cat_cols
                        )

In [52]:
def readStoredModels(pathToReadFrom, fileList):
  model_dfs = []
  os.chdir(pathToReadFrom)
  for file in fileList:
    df = pd.read_pickle(file)
    model_dfs.append(df)
  return model_dfs


In [54]:
os.chdir(modelsPath)
files = os.listdir()
file_list = [file for file in files if ("deg_a_ck" in file) or ("eig_a_ck" in file) or ("pr_a_ck" in file) or ("nf_a_ck" in file)]


In [55]:
models = readStoredModels(modelsPath, file_list)

In [56]:
# Merge models
for i, model in enumerate(models):
  if i == 0:
    df = model
  else:
    df = pd.merge(df, model, left_index=True, right_index=True)  

In [57]:
df

,deg_a_ck_b,eig_a_ck_b,pr_a_ck_b,nf_a_ck_b,deg_a_ck_a_p_b,eig_a_ck_a_p_b,pr_a_ck_a_p_b,nf_a_ck_a_p_b
a,,,,,,,,
1.0,1.000000,0.033797,0.000691,0.006367,0.0,0.009168,0.000247,0.015725
2.0,0.999753,0.033792,0.000691,0.007228,0.0,0.023903,0.000247,0.015725
3.0,0.999505,0.033788,0.000691,0.005311,0.0,0.018677,0.000247,0.015725
4.0,0.997774,0.033758,0.000689,0.001668,0.0,0.019403,0.000247,0.015725
5.0,0.997032,0.033736,0.000689,0.005279,0.0,0.022181,0.000247,0.015725
...,...,...,...,...,...,...,...,...
4040.0,0.269849,0.012466,0.000204,0.003587,0.0,-0.008227,0.000247,0.015725
4041.0,0.232501,0.010928,0.000181,-0.033325,0.0,-0.023879,0.000247,0.015725
4042.0,0.225822,0.010539,0.000177,-0.016581,0.0,0.011236,0.000247,0.015725


In [59]:
# Remove constant columns
df = df.loc[:, (df != df.iloc[0]).any()]
df.head() 

In [60]:
# Save dataframe:
os.chdir(dataPath)
df.to_pickle("word2vec.pkl")


In [61]:
# Load word2vec
os.chdir(dataPath)
df_word= pd.read_pickle("word2vec.pkl")
df_word.head()
df_word.tail()

,deg_a_ck_b,eig_a_ck_b,pr_a_ck_b,nf_a_ck_b,eig_a_ck_a_p_b
a,,,,,
1.0,1.000000,0.033797,0.000691,0.006367,0.009168
2.0,0.999753,0.033792,0.000691,0.007228,0.023903
3.0,0.999505,0.033788,0.000691,0.005311,0.018677
4.0,0.997774,0.033758,0.000689,0.001668,0.019403
5.0,0.997032,0.033736,0.000689,0.005279,0.022181


In [63]:
os.chdir(dataPath)
df_word.to_csv('vectors.tsv', sep="\t", index = False, header = False)

In [64]:
metadata = ast.literal_eval(re.search('({.+})', word2index).group(0))

In [ ]:
str_list = list(metadata.keys())
str_list

In [66]:
import sys
orig_stdout = sys.stdout
f = open('metadata.tsv', 'w')
sys.stdout = f

print(' \n'.join(str_list))

sys.stdout = orig_stdout
f.close()
sys.stdout = orig_stdout

In [ ]:
######## I am done ################3